In [24]:
from py2neo import Graph
from pandas import DataFrame
from pprint import pprint

# Test

In [2]:
graph = Graph(host="localhost://7474", auth=("neo4j", "123456"))

In [27]:
query = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = 'local'
        WITH re,COUNT(r) AS number,SUM(a.restaurant_sentiment) AS senti      
        
        RETURN re.name, senti/number AS average_senti, number
        ORDER BY average_senti DESC

"""

a=graph.run(query).data()
a

[{'re.name': '/scaled-by-ah-hua-kelong',
  'average_senti': 0.9188084357738098,
  'number': 84},
 {'re.name': '/azmi-chappati-thye-chong-restaurant',
  'average_senti': 0.9025676618888888,
  'number': 18},
 {'re.name': '/red-sparrow',
  'average_senti': 0.8993222303888889,
  'number': 36},
 {'re.name': '/chef-kang', 'average_senti': 0.8990597711666669, 'number': 54},
 {'re.name': '/2RflyYEv', 'average_senti': 0.8904110564999997, 'number': 24},
 {'re.name': '/enjoy-eating-house-and-bar',
  'average_senti': 0.8875170693916665,
  'number': 120},
 {'re.name': '/han-kee-fish-soup',
  'average_senti': 0.885581858611111,
  'number': 36},
 {'re.name': '/mings-prawn-noodle',
  'average_senti': 0.885323691611111,
  'number': 18},
 {'re.name': '/obba-jjajang',
  'average_senti': 0.8815650229027774,
  'number': 72},
 {'re.name': '/su-korean-cuisine',
  'average_senti': 0.8783748867857144,
  'number': 42},
 {'re.name': '/pig-organ-soup-jalan-besar',
  'average_senti': 0.8779657240000003,
  'number'

In [72]:
query_b = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = 'chickenrice'
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY average_senti DESC
        """
b=graph.run(query_b).data()
b

[{'re.name': '/power-coffeehouse', 'average_senti': 0.980081348, 'number': 1},
 {'re.name': '/bedok-85-market',
  'average_senti': 0.9783189109999999,
  'number': 1},
 {'re.name': '/tanglin-halt-commonwealth-drive-food-centre-2',
  'average_senti': 0.9696601659999999,
  'number': 1},
 {'re.name': '/ghim-moh-market-food-centre',
  'average_senti': 0.959340527,
  'number': 1},
 {'re.name': '/albert-centre-market-food-centre-queen-street-market',
  'average_senti': 0.9526417290000001,
  'number': 1},
 {'re.name': '/redhill-market-food-centre',
  'average_senti': 0.943759565,
  'number': 1},
 {'re.name': '/victor-famous-fried-chicken-wing-rice-veerasamy-road',
  'average_senti': 0.9385110145,
  'number': 2},
 {'re.name': '/hambaobao', 'average_senti': 0.926456893, 'number': 1},
 {'re.name': '/roast-paradise', 'average_senti': 0.920202953, 'number': 1},
 {'re.name': '/geylang-claypot-rice',
  'average_senti': 0.916828027,
  'number': 1},
 {'re.name': '/malaysia-boleh', 'average_senti': 0.90

'/scaled-by-ah-hua-kelong'

# Search query functions

In [112]:
def search_restaurant_by_food(graph,text,sort):
    result=[]
    key_words = text
    
    query_rating = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = {term}
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY average_senti DESC
        """
    query_popularity = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = {term}
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY number DESC
        """
    
    if sort == 'number_of_reviews':
        query = query_popularity

    if sort == 'rating':
        query = query_rating
        
    ans = graph.run(query,term=text).data()
    
    if ans:
        for item in ans:
            result.append(item[re.name])
        return result
    
    if not ans:
        key_words = key_words.split()
        result={}
        
        for i in range(len(key_words)):
            temp = graph.run(query, term=key_words[i]).data()
            result[key_words[i]]=[]
            for item in temp:
                result[key_words[i]].append(item[re.name])
                
        if not result:
            return ['No records found']
        else:
            return result
        

def search_restaurant_by_cusine(text,sort):
    result=[]
    
    query_rating = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = {term}
        WITH re,COUNT(r) AS number,SUM(a.restaurant_sentiment) AS senti      
        
        RETURN re.name, senti/number AS average_senti
        ORDER BY average_senti DESC

    """
    
    query_popularity = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = {term}
        WITH re,COUNT(r) AS number        
        RETURN re.name,number
        ORDER BY number DESC

    """
    if sort == 'number_of_reviews':
        query = query_popularity
    
    if sort == 'rating':
        query = query_rating
        
    result = graph.run(query, term=text).data()
    if result == []:
        return ['No records found']
    else: 
        return result
    

